In [2]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
package_path = ['/content/drive/MyDrive/Packages', '/content/drive/MyDrive/github/gg_job_search', '/content/drive/MyDrive/github/gg_job_search/src']
sys.path.extend(package_path)
#!pip install --target=$package_path cupy-cuda102

Mounted at /content/drive


## Load libraries

In [1]:
import sys

In [8]:
sys.path.insert(0, '../')
sys.path

['../',
 '../',
 '/home/axel/Documents/gg_job_search/notebooks',
 '/home/axel/anaconda3/envs/gg_job_search/lib/python311.zip',
 '/home/axel/anaconda3/envs/gg_job_search/lib/python3.11',
 '/home/axel/anaconda3/envs/gg_job_search/lib/python3.11/lib-dynload',
 '',
 '/home/axel/anaconda3/envs/gg_job_search/lib/python3.11/site-packages']

In [4]:
import pandas as pd
import numpy as np

import sqlite3

# custom functions
from src.preprocessing import preprocess as pp

from config import DB_PATH

## Load data

In [5]:
conn = sqlite3.connect(DB_PATH)
c = conn.cursor()

c.execute('''

   SELECT * FROM deduplicated_data;
          
''')

rows = c.fetchall()

columns = [column[0] for column in c.description]

all_data = pd.DataFrame(rows, columns=columns)

In [42]:
predicted_salaries = pd.read_csv('/home/axel/Documents/gg_job_search/data/qa_pred_salary.csv')

#pred_df = predicted_salaries.copy()
#all_df = all_data.copy()

all_df = pd.read_csv('/home/axel/Documents/gg_job_search/data/gg_job_search_all_RAW.csv')

In [44]:
predicted_salaries

,id,prediction_text,logit_score,context,lang_labels,schedule_type,date_time
0,0,Pas de salaire précisé,7.586216,fifty-five recherche ...\n\ndescription\ndata ...,fr,NaN,2023-02-17 11:50:44.355114
1,1,Pas de salaire précisé,8.930310,contexte : dans le cadre de plusieurs projets ...,fr,NaN,2023-02-17 11:50:44.355114
2,2,essilorluxottica,-0.181155,quelles sont les missions ?\n\nvous\navez beso...,fr,NaN,2023-02-17 11:50:44.355114
3,3,Pas de salaire précisé,7.904837,consept informatique recherche ...\n\nintegre(...,fr,NaN,2023-02-17 11:50:44.355114
4,5,Pas de salaire précisé,7.298598,"presentation de la societe\n\nlance en 2021, t...",fr,NaN,2023-02-17 11:50:44.355114
...,...,...,...,...,...,...,...
16911,17168,"$91,587 to $111,812",0.147631,job description:\ndata engineer\n\nwelcome to ...,en,a plein temps,2023-05-18 22:28:20.808374
16912,17174,salary nc,-2.053096,eb partners aims strengthening his solutions r...,en,a plein temps,2023-05-18 22:28:22.947335
16913,17183,salary nc,-2.119184,about unilever unilever is one of the world`s ...,en,a plein temps,2023-05-18 22:28:30.440350
16914,17184,you will work in multi-disciplinary environmen...,-4.312231,qualifications master’s degree in quantitative...,en,a plein temps,2023-05-18 22:28:30.440350


In [ ]:
# preprocessing
all_df = pp.basic_prepro(all_df)

## Get salaries from extensions

In [12]:
# split list to get salaries only
extracted_salaries = pd.Series(all_df.extensions.str.split(', ', expand=True)[1], index=all_df.index)
extracted_salaries.dropna(axis=0, inplace=True)

In [13]:
# remove rows  w/o salary info
extracted_salaries.loc[extracted_salaries.str.contains("plein temps|temps partiel|stage|prestataire", regex=True, case=False)] = np.NaN
extracted_salaries.dropna(axis=0, inplace=True)

In [14]:
# get salary currency / period / status
extensions_salary_period = pp.get_salary_period(extracted_salaries).to_frame('salary_period').set_index(extracted_salaries.index)
extensions_salary_currency = pp.get_salary_currency(extracted_salaries).to_frame('salary_currency').set_index(extracted_salaries.index)
extensions_salary_status = pp.get_salary_status(extracted_salaries).to_frame('salary_status').set_index(extracted_salaries.index)

# concat salaries infos
ext_slr_infos = pd.concat([extensions_salary_period, extensions_salary_currency, extensions_salary_status], axis=1)

In [15]:
# per year
extracted_salaries = extracted_salaries.str.replace(r'(\d{2}|\d{3})(?:\\xa0k\\xa0€ a |\\xa0k\\xa0\$us a )(\d{2}|\d{3})(?:\\xa0k\\xa0€ par an|\\xa0k\\xa0\$us par an)', r'\g<1>000 a \g<2>000', regex=True)
extracted_salaries = extracted_salaries.str.replace(r'(\d{1}|\d{2}|\d{3})(?:,)(\d{1}|\d{2}|\d{3})(?:\\xa0k\\xa0€)( a )(\d{1}|\d{2}|\d{3})(?:,)(\d{1}|\d{2}|\d{3})(?:\\xa0k\\xa0€ par mois)', r'\g<1>\g<2>00\3\g<4>\g<5>00', regex=True)

extracted_salaries = extracted_salaries.str.replace(r'(\d{2}|\d{3})(?:\\xa0€ a )(\d{2}|\d{3})(?:\\xa0€ par an)', r'\g<1>000 a \g<2>000', regex=True)

extracted_salaries = extracted_salaries.str.replace(r'(\d{2}|\d{3})(?:\\u202f)(\d+)(?:,\d{2}|)(?:\\xa0)(?:\$us|€)( a )(\d{2}|\d{3})(?:\\u202f)(\d+)(?:,\d{2}|)(?:\\xa0)(?:\$us|€)( par an)', r'\1\2\3\4\5', regex=True)

extracted_salaries = extracted_salaries.str.replace(r'(\d{2}|\d{3})(?:\\xa0k\\xa0)(?:€|\$us)(?: par an)', r'\g<1>000', regex=True)

extracted_salaries = extracted_salaries.str.replace(r'(\d{2}|\d{3})\\u202f(\d{3})\\xa0€ par an', r'\1\2', regex=True)
extracted_salaries = extracted_salaries.str.replace(r'(\d{1})\\u202f(\d{3})(?:,\d{2}\\xa0€)', r'\1\2', regex=True)

extracted_salaries = extracted_salaries.str.replace(r'(\d{2})(?:,)(\d{1})(?:\\xa0k\\xa0€)', r'\g<1>\g<2>00', regex=True)

# per month
extracted_salaries = extracted_salaries.str.replace(r'(\d{1})(?:\\u202f)(\d+)(?:,\d{2}|)(?:\\xa0)(?:\$us|€) par mois', r'\1\2', regex=True)
extracted_salaries = extracted_salaries.str.replace(r'(\d{1})\\u202f(\d{3})(?:,\d{2}\\xa0€)', r'\1\2', regex=True)

extracted_salaries = extracted_salaries.str.replace(r'(\d{1})(?:,)(\d{1})(?:\\xa0k\\xa0€)', r'\1\g<2>00', regex=True)
extracted_salaries = extracted_salaries.str.replace(r'(\d{1})(?:\\xa0k\\xa0€)', r'\g<1>000', regex=True)

extracted_salaries = extracted_salaries.str.replace(r'(\d{1})(?:\\u202f)(\d{3})(?:\\xa0€)( a)', r'\1\2\3', regex=True)

extracted_salaries = extracted_salaries.str.replace(r'(\d{3})(?:\\xa0€) (par| a)', r'\1 \2', regex=True)
extracted_salaries = extracted_salaries.str.replace(r'(\d{3})(?:,)(?:\d{2})(?:\\xa0€)( par mois| a)', r'\1', regex=True)

# per day
extracted_salaries = extracted_salaries.str.replace(r'(\d{2}|\d{3})(?:\\xa0€)( a )(\d{2}|\d{3})(?:\\xa0€) par jour', r'\1\2\3', regex=True)
extracted_salaries = extracted_salaries.str.replace(r'(\d{2}|\d{3})(?:\\xa0€ par jour)', r'\1', regex=True)

# per hours
extracted_salaries = extracted_salaries.str.replace(r"(\d{1})(?:\\u202f\d{3})(?:\\xa0€)( l'heure)", r'\1', regex=True)


# miscellanous
extracted_salaries = extracted_salaries.str.replace(r"(\d{2})(?:,)(\d{1})(?:\\xa0k\\xa0\$us)( a)", r'\1\g<2>00\3', regex=True)

extracted_salaries = extracted_salaries.str.replace(r"(\d{1})(?:\\u202f\d{3})(?:\\xa0€)( l'heure)", r'\1', regex=True)

extracted_salaries = extracted_salaries.str.replace(r"(\d{2}|\d{1})(,\d{1}|,\d{2})(?:\\xa0€)", r'\1\2', regex=True)

extracted_salaries = extracted_salaries.str.replace(r"(\d{1})(?:\\xa0€)(?: par mois)", r'\g<1>000', regex=True)

extracted_salaries = extracted_salaries.str.replace(r"(\d{2}|\d{3})(?:\\xa0€)", r'\g<1>', regex=True)

extracted_salaries = extracted_salaries.str.replace(r",", r'.', regex=True)


In [16]:
# keep digits only
slr_df = extracted_salaries.str.split(' ', expand=True)
slr_df = slr_df.applymap(lambda x: pp.rm_non_digits(x))

# report all values into 2 columns
slr_df = slr_df.fillna(method='bfill', axis=1).iloc[:,0:2].rename(columns={0:'lower_bound', 1:'upper_bound'})

extensions_discrete_salaries = slr_df[slr_df.upper_bound.isnull()].iloc[:,0]
extensions_range_salaries = slr_df.drop(extensions_discrete_salaries.index, axis=0)
extensions_meaned_salaries = extensions_range_salaries[['lower_bound', 'upper_bound']].mean(axis=1)

# concat all discrete salaries
extensions_salaries = pd.concat([extensions_discrete_salaries, extensions_meaned_salaries], axis=0).to_frame('salaries')

# merge w/ currency, period and status infos
extensions_salaries = extensions_salaries.merge(ext_slr_infos, left_index=True, right_index=True, how='left')

## Cleaning french predictions

In [45]:
# get french records
pred_fr = pp.get_lang_records(predicted_salaries, 'fr').set_index('id')

In [47]:
pred_fr['id'] = pred_fr.index

In [48]:
pred_fr

,prediction_text,logit_score,context,lang_labels,schedule_type,date_time,id
id,,,,,,,
0,Pas de salaire précisé,7.586216,fifty-five recherche ...\n\ndescription\ndata ...,fr,NaN,2023-02-17 11:50:44.355114,0
1,Pas de salaire précisé,8.930310,contexte : dans le cadre de plusieurs projets ...,fr,NaN,2023-02-17 11:50:44.355114,1
2,essilorluxottica,-0.181155,quelles sont les missions ?\n\nvous\navez beso...,fr,NaN,2023-02-17 11:50:44.355114,2
3,Pas de salaire précisé,7.904837,consept informatique recherche ...\n\nintegre(...,fr,NaN,2023-02-17 11:50:44.355114,3
5,Pas de salaire précisé,7.298598,"presentation de la societe\n\nlance en 2021, t...",fr,NaN,2023-02-17 11:50:44.355114,5
...,...,...,...,...,...,...,...
17180,jusqu'a 60k,3.181781,data engineer dans une edtech - cdi - paris - ...,fr,a plein temps,2023-05-18 22:28:26.923039,17180
17181,600 millions d'euros,3.439713,talan est un cabinet de conseil en innovation ...,fr,a plein temps,2023-05-18 22:28:26.923039,17181
17182,Pas de salaire précisé,13.730790,environnement de travail au sein d’une equipe ...,fr,a plein temps,2023-05-18 22:28:26.923039,17182


In [57]:
pred_text = pred_fr['prediction_text']
pred_context = pred_fr['context']

# get salary currency / period / status
pred_salary_period = pp.get_salary_period(pred_text).to_frame('salary_period')
pred_salary_currency = pp.get_salary_currency(pred_text).to_frame('salary_currency')
pred_salary_status = pp.get_salary_status(pred_text).to_frame('salary_status')

# concat salaries infos
pred_slr_infos = pd.concat([ pred_salary_period, pred_salary_currency, pred_salary_status], axis=1).set_index(pred_fr.id)

In [59]:
pred_slr_infos = pd.merge(pred_slr_infos, pred_context, on='id')


In [60]:
# remove prediction w/ "Pas de salaire précisé" in it
pred_fr = pred_fr[~pred_fr['prediction_text'].str.contains(r'pas de salaire', case=False)]

# rm records w/o numbers in it
with_numbers = pred_fr[pred_fr['prediction_text'].str.contains(r'[0-9]', regex=True)]

# sort values
sorted_preds = with_numbers.sort_values(by='logit_score', ascending=False)

# remove text w/o ref to currency / salary status (=> low density of salary among predictions)
hard_to_get_salaries = sorted_preds[~sorted_preds.prediction_text.str.contains(r'k|\beur|¤|£|€|\bke\b|\btjm\b', regex=True)]
easy_salaries = sorted_preds.drop(hard_to_get_salaries.index, axis=0)

# remove longest text 
easy_salaries['pred_text_len'] = easy_salaries.prediction_text.str.len()
long_pred_text = easy_salaries[easy_salaries['pred_text_len'] >= 38]
easy_salaries = easy_salaries.drop(long_pred_text.index, axis=0)

# remove text w/ billions ref in it
fund_raise = easy_salaries[easy_salaries.prediction_text.str.contains('m€|millions|milliards|md€|m €|md €|[0-9]m|milliard|md')]
easy_salaries = easy_salaries.drop(fund_raise.index, axis=0)

# remove miscellanous words not refering to salaries
other_words = easy_salaries[easy_salaries.prediction_text.str.contains('bac|ingenieur|collaborateurs')]
easy_salaries = easy_salaries.drop(other_words.index, axis=0)

hard_to_get_salaries = pd.concat([hard_to_get_salaries, long_pred_text, fund_raise, other_words], axis=0)

In [61]:
df = easy_salaries.copy()

df.prediction_text = df.prediction_text.str.replace('entre', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('et| et ', ' a ', regex=True)
df.prediction_text = df.prediction_text.str.replace(r' - |-|–| – ', ' a ', regex=True)
df.prediction_text = df.prediction_text.str.replace(r' \bet\b |\bet\b', ' a ', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'([0-9]) (/) ([0-9])', r'\1 a \3', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'([0-9])(/)([0-9])', r'\1 a \3', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'([a-z]) (/) ([a-z])|([a-z€])(/)([a-z€])', r'\4 par \6', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'\+', '', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'\(', '', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'\?', '', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'\)', '', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'\|~', '', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'de', '', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'mensuel', 'par mois', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'annuel', 'par an', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'(?:euro?s?|eur)\b', ' € ', regex=True)
df.prediction_text = df.prediction_text.str.replace('(\d{1})( )(\d{3})', r'\1\3', regex=True)
df.prediction_text = df.prediction_text.str.replace("(?P<first>\d{2}|\d{2}k|\d{2} k)\s+(?P<second>[1-9]\d{1} k|[1-9]\d{1}k|[1-9]\d{1}\b)", "\g<first> a \g<second>", regex=True)
df.prediction_text = df.prediction_text.str.replace('\.(\d{3})', r'\1', regex=True)
df.prediction_text = df.prediction_text.str.replace('(?P<first>[0-9]) (?P<second>k)', '\g<first>\g<second>', regex=True)
df.prediction_text = df.prediction_text.str.replace('\.\d{2}|\.\d{1}', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('\.', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('(?P<first>[0-9])(?P<second> a )(?P<third>\d{2}k|\d{3}k)', '\g<first>k\g<second>\g<third>', regex=True)
df.prediction_text = df.prediction_text.str.replace('(?P<first>,\d{1}|,\d{2})(?P<second>k)', '\g<second>', regex=True)
df.prediction_text = df.prediction_text.str.replace('(,\d{2})( €|€)', '€', regex=True)
df.prediction_text = df.prediction_text.str.replace('\n', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('¤', '€', regex=True)
df.prediction_text = df.prediction_text.str.replace('/ an|/an| /an| / an', ' par an', regex=True)
df.prediction_text = df.prediction_text.str.replace('/h\b|/ h\b| /h\b| / h\b', ' par heure', regex=True)
df.prediction_text = df.prediction_text.str.replace('/ mois', 'par mois', regex=True)
df.prediction_text = df.prediction_text.str.replace('a partir', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('(?P<first>k)(?P<second>\d{2})', '\g<second>\g<first>', regex=True)
df.prediction_text = df.prediction_text.str.replace('(?P<first>,|,\s+)(?P<second>\d{3})', '\g<second>', regex=True)
df.prediction_text = df.prediction_text.str.replace('\~', '', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'([0-9])( a )(\d{2})\b', r'\1k\2\3k', regex=True)
df.prediction_text = df.prediction_text.str.replace('net', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('brut', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('par an|per year|annuel', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('par mois|/13 mois', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('par jour|tjm|/ jour', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('heure|par h|par heure', '', regex=True)
df.prediction_text = df.prediction_text.str.replace('€', '', regex=True)
df.prediction_text = df.prediction_text.str.replace("jusqu'a", '', regex=True)
df.prediction_text = df.prediction_text.str.replace('k', '000', regex=True)
df.prediction_text = df.prediction_text.str.replace('(?P<first>\d{5})(?P<second>a)(?P<third>\d{5})', '\g<first> \g<second> \g<third>', regex=True)
df.prediction_text = df.prediction_text.str.replace(',', '.', regex=True)
df.prediction_text = df.prediction_text.str.replace('(?P<first>[0-9])(?P<second>e)', '\g<first>', regex=True)
df.prediction_text = df.prediction_text.str.replace(r'(?P<first>[0-9][0-9]000)(?P<second>|\s*)(?P<third>[0-9][0-9]000)', r'\g<first> a \g<third>', regex=True)

In [64]:
# keep digits only
pred_slr_df = df.prediction_text.str.split(' ', expand=True)
pred_slr_df = pred_slr_df.applymap(lambda x: pp.rm_non_digits(x))

# report all values into 2 columns
pred_slr_df = pred_slr_df.fillna(method='bfill', axis=1).iloc[:,0:2].rename(columns={0:'lower_bound', 1:'upper_bound'})

In [66]:

discrete_pred_salaries = pred_slr_df[pred_slr_df.upper_bound.isnull()].iloc[:,0]
range_pred_salaries = pred_slr_df.drop(discrete_pred_salaries.index, axis=0)
meaned_pred_salaries = range_pred_salaries[['lower_bound', 'upper_bound']].mean(axis=1)

# concat all discrete salaries
#prediction_salaries = pd.concat([discrete_pred_salaries, meaned_pred_salaries], axis=0).to_frame('salaries')

# merge w/ currency, period and status infos
prediction_salaries = pred_slr_df.merge(pred_slr_infos, on='id', how='left')

In [75]:
prediction_salaries.to_csv('labeled_data.csv')

In [72]:
pd.options.display.max_colwidth = 5000
pred_only = all_df.iloc[prediction_salaries.index]

#pred_only.loc[pred_only.description.str.len() < 1000]['description'].to_csv('/content/drive/MyDrive/github/gg_job_search/data/sal_data.csv', index=False)

In [41]:
#


#description = predicted_salaries.context.iloc[pred_slr_df.index]

#pd.concat([pred_slr_df, description], axis=1)

prediction_salaries

,lower_bound,upper_bound,salary_period,salary_currency,salary_status
id,,,,,
8690,45000.0,60000.0,nan,euros,brut
12316,36000.0,36000.0,nan,euros,brut
7162,1000.0,NaN,mois,euros,brut
12061,36000.0,36000.0,nan,euros,brut
5763,38000.0,40000.0,nan,euros,brut
...,...,...,...,...,...
8574,500.0,600.0,nan,euros,brut
10279,3.0,NaN,nan,euros,brut
10282,3.0,NaN,nan,euros,brut


0## Concat all salaries + add schedule_type

In [55]:
# concat salaries from extensions column and predicted salaries
#all_salaries = pd.concat([extensions_salaries, prediction_salaries], axis=0)
all_salaries = extensions_salaries.combine_first(prediction_salaries)

# add schedule-type column from all_df
#all_salaries = all_salaries.merge(all_df.schedule_type, left_index=True, right_index=True, how='left')

In [59]:
prediction_salaries

,salaries,salary_period,salary_currency,salary_status
id,,,,
7162,1000.00,mois,euros,brut
5080,1000.00,mois,euros,brut
7206,1000.00,mois,euros,brut
3061,2241.00,nan,euros,brut
9306,2241.00,nan,euros,brut
...,...,...,...,...
7318,42500.00,nan,euros,brut
17158,67500.00,nan,euros,brut
5328,43000.00,nan,euros,brut


## Filter out incoherent salaries

In [46]:
# Set float format to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format

# divise by 10 salaries over 200k
all_salaries.loc[all_salaries.salaries > 200000, 'salaries'] = all_salaries.loc[all_salaries.salaries > 200000, 'salaries']/ 10

# remove salaries still over 200k
over_200k = all_salaries.loc[all_salaries.salaries > 200000, 'salaries']
all_salaries.drop(over_200k.index, axis=0, inplace=True)

In [ ]:
all_df[all_df.index == 8642][['title', 'description', 'extensions']]
#all_df.loc[[2890, 3921, 5125, 8578, 12800, 12800]]

In [3]:
# under 17500 per year 
all_salaries.drop(all_salaries[(all_salaries.salaries < 17500) & (all_salaries.salary_period == 'an')].index, axis=0, inplace=True)

# under 600 per month
all_salaries.drop(all_salaries[(all_salaries.salaries < 600) & (all_salaries.salary_period == 'mois')].index, axis=0, inplace=True)

# under 135 per day
all_salaries.drop(all_salaries[(all_salaries.salaries <135) & (all_salaries.salary_period == 'jour')].index, axis=0, inplace=True)

# under 9.76 or over 60 per hour
all_salaries.drop(all_salaries[((all_salaries.salaries < 9.76) | (all_salaries.salaries > 60)) & (all_salaries.salary_period == 'heure')].index, axis=0, inplace=True)

# under 9.76
all_salaries.drop(all_salaries[(all_salaries.salaries < 9.76 )].index, axis=0, inplace=True)

all_salaries.drop(all_salaries[(all_salaries.salary_period == 'nan') & (all_salaries.schedule_type.isin(["a plein temps", "a temps partiel"])) & (all_salaries.salaries > 11 ) & (all_salaries.salaries < 100 )].index, axis=0, inplace=True)
all_salaries.drop(all_salaries[(all_salaries.salary_period == 'nan') & (all_salaries.schedule_type.isin(["a plein temps", "a temps partiel"])) & (all_salaries.salaries > 450 ) & (all_salaries.salaries <= 550 )].index, axis=0, inplace=True)

# probably per day / prestataire
all_salaries.loc[(all_salaries.salary_period == 'nan')& (all_salaries.salaries > 250 ) & (all_salaries.salaries < 600 ) & (all_salaries.schedule_type.isin(["a plein temps", "a temps partiel"])), 'salary_period'] = 'jour'

#all_salaries.loc[(all_salaries.salary_period == 'nan')& (all_salaries.salaries > 250 ) & (all_salaries.salaries < 600 ) & (all_salaries.schedule_type.isin(["a plein temps", "a temps partiel"]))]

all_salaries

NameError: ignored

In [ ]:
all_df.iloc[all_salaries.index]['description'][25]

## Export

In [ ]:
#salary_frdf.to_csv('/content/drive/MyDrive/github/gg_job_search/data/cleaned_salaries.csv', index=False)#, header=False, mode='a')